In [24]:
import pandas as pd
import numpy as np
from scipy import stats

'''
ALDH1 : 319
PKM2 : 36
VDR : 52
'''
for dataset in ['ALDH1', 'PKM2', 'VDR']:
    dir1 = {'ALDH1': f'75. fp+mf+um_attention_2', 
            'PKM2': f'75. fp+mf+um_attention_2', 
            'VDR': f'75. fp+mf+um_attention_2'}
    file1 = {'ALDH1': f'75. {dataset}_exploitation_mlp_fp+mf+um_0_nonorm_512_256', 
            'PKM2': f'75. {dataset}_exploitation_mlp_fp+mf+um_0_nonorm_512_64', 
            'VDR': f'75. {dataset}_exploitation_mlp_fp+mf+um_0_nonorm_512_128_layer2'}
    # dir2 = {'ALDH1': f'81. fp+mf+um_attention_seq_kl_T1_ratio0.25', 
    #         'PKM2': f'81. fp+mf+um_attention_seq_kl_T1_ratio0.25', 
    #         'VDR': f'81. fp+mf+um_attention_seq_kl_T1_ratio0.5'}
    # file2 = {'ALDH1': f'81. {dataset}_exploitation_mlp_fp+mf+um_0_nonorm_512_256_layer2_cycle1_lambdaVar', 
    #         'PKM2': f'81. {dataset}_exploitation_mlp_fp+mf+um_0_nonorm_1024_512_cycle1_lambdaVar', 
    #         'VDR': f'81. {dataset}_exploitation_mlp_fp+mf+um_0_nonorm_512_64_cycle2_lambdaVar'}
    dir2 = {'ALDH1': f'82. fp+mf+um_attention_ucb_1', 
            'PKM2': f'82. fp+mf+um_attention_ucb_1', 
            'VDR': f'82. fp+mf+um_attention_ucb_1'}
    file2 = {'ALDH1': f'82. {dataset}_exploitation_mlp_fp+mf+um_0_nonorm_512_256_beta1', 
            'PKM2': f'82. {dataset}_exploitation_mlp_fp+mf+um_0_nonorm_512_256_layer2_beta0.5', 
            'VDR': f'82. {dataset}_exploitation_mlp_fp+mf+um_0_nonorm_512_128_layer2_beta0.75'}
    acqf = 'exploitation'
    arch1 = 'mlp'
    num_hit1 = []
    num_hit2 = []

    data1 = pd.read_csv(f'../results_0924/{dir1[dataset]}/{file1[dataset]}/{file1[dataset]}.csv')
    data2 = pd.read_csv(f'../results_1014/{dir2[dataset]}/{file2[dataset]}/{file2[dataset]}.csv')

    for i in range(16):
        data = data1[(data1['acquisition_method'] == acqf) & (data1['architecture'] == arch1) & (data1['dataset'] == dataset) \
                & (data1['screen_cycle'] == i)]
        num_hit1.append(list(data['hits_discovered']))

    for i in range(16):
        data = data2[(data2['acquisition_method'] == acqf) & (data2['architecture'] == arch1) & (data2['dataset'] == dataset) \
                & (data2['screen_cycle'] == i)]
        num_hit2.append(list(data['hits_discovered']))

    num_hit1 = np.array(num_hit1).T
    num_hit2 = np.array(num_hit2).T
    print(num_hit1[:,-1], num_hit1[:,-1].mean())
    # num_hit1[3,-1] = 30
    print(num_hit2[:,-1], num_hit2[:,-1].mean())

    print(num_hit1[:,-1].std())
    print(num_hit2[:,-1].std())

    t_stat, p_value = stats.ttest_rel(num_hit1, num_hit2)
    # print("t-statistic:", t_stat)
    print("p-value:", p_value)

    t_stat, p_value = stats.ttest_rel(num_hit1[:,-1], num_hit2[:,-1])
    # print("t-statistic:", round(t_stat,4))
    print("p-value:", round(p_value,10))

     

[298 313 313 268 301] 298.6
[349 335 329 273 293] 315.8
16.475436261295176
28.27295527531567
p-value: [       nan 0.5291333  0.51764812 0.45945266 0.48585384 0.51710249
 0.46832059 0.29498379 0.22717688 0.47064922 0.58512811 0.64316872
 0.95142157 0.45299349 0.33979483 0.15655335]
p-value: 0.1565533523
[34 28 25  8 35] 26.0
[33 35 27 32 46] 34.6
9.736529155710468
6.280127387243033
p-value: [       nan 0.37390097 0.09930068 0.43096867 0.66508573 0.36907705
 0.7201997  0.69889314 0.57100219 0.78238137 0.94062221 0.40558273
 0.18407403 0.13103213 0.12495216 0.12018776]
p-value: 0.1201877638
[29 43 42 32 60] 41.2
[45 47 51 34 67] 48.8
10.870142593360953
10.703270528207721
p-value: [       nan 0.47662067 0.31685485 0.5354236  0.68845709 0.68270603
 0.36016899 0.7007472  0.66085822 1.         0.47662067 0.12835349
 0.01751619 0.04509118 0.10454    0.03486348]
p-value: 0.0348634794


In [22]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import BRICS
from collections import defaultdict

def brics_decomp(smiles_list, label_list):
    result = defaultdict(list)
    set_result = defaultdict(list)

    for smi, label in zip(smiles_list, label_list):
        brics_list = []
        mol = Chem.MolFromSmiles(smi)
        fragments = BRICS.BRICSDecompose(mol)
        for frag in fragments:
            brics_list.append(frag)
        result[label].append(brics_list)
        set_result[label].extend(brics_list)

    # 중복 제거
    total_frag_set = list(set(set_result[0]) | set(set_result[1]))

    # fragment별 positive / negative 개수 세기
    frag_stats = []
    for frag in total_frag_set:
        pos_count = sum(frag in frags for frags in result[1])
        neg_count = sum(frag in frags for frags in result[0])
        total = pos_count + neg_count
        ratio = pos_count / total if total > 0 else 0
        frag_stats.append({
            'fragment': frag,
            'positive_count': pos_count,
            'negative_count': neg_count,
            'positive_ratio': ratio
        })

    df = pd.DataFrame(frag_stats)
    return df.round(5)

brics_dict = {}
for dataset in ['ALDH1', 'VDR']:
    data1 = pd.read_csv(f'../data/{dataset}/original/screen.csv')
    positive = data1[data1['y'] == 1]
    smiles_list = data1['smiles']
    label_list = data1['y']

    df = brics_decomp(smiles_list, label_list)
    df.to_csv(f'./result_brics_{dataset}.csv')

In [ ]:
    for dataset in ['ALDH1', 'PKM2', 'VDR']:
        data = pd.read_csv(f'./result_brics_{dataset}.csv')
    data